# Advanced Pytest

## Pytest Fixtures

Fixtures are a good idea for 2 main reasons:

1. They keep dependencies separate to the tests.
2. They encourage the reuse of fixtures for functions with similar requirements.

Each fixture should be designed with 3 stages in mind:

1. setup
2. assert
3. teardown

In [44]:
import pytest
import os
from unittest.mock import call


In [20]:
def goodie(csv):
    with open(csv, "r") as f:
        txt = f.read()
        f.close()
    lines = txt.splitext()
    lines = [l for l in lines if "good" in l]
    return lines

In [27]:
@pytest.fixture
def super_file(tmpdir):
    # setup
    contents = """name,polarity,\nspiderman,good,\nvenom,bad"""
    # append the filepth with the tempdir ref
    pth = tmpdir.join("superheroes.csv")
    with open(pth, "w") as f:
        f.write(contents)
        f.close()
    # pytest no longer supports use of yield as shown in course
    yield pth
    # tmpdir will handle the teardown phase, removing the csv from disk

In [30]:
def test_goodie(super_file):
    expected = "spiderman,good"
    found = goodie(super_file)
    assert found == expected


***

## Mocking

This section was really dense, further exemplification required.

The course depicts an integration test where 2 dependencies are used. The first helper
function has a bug. The course states that **the point of testing this function should
not be to find bugs in its dependencies.** This needs further exploration. Mocking use
cases I have encountered in the past tend to simulate the expected responses of external
resources, for example API calls, url requests, os-related behaviours (pretend to be 
unix-like when running on Windows etc). 

Instead of testing the dependency when checking the behaviour of the wrapper, the course
mocks the result of the dependency with MagicMock, ensuring the wrapper's test passes
even though a bug exists in the dependency. While this would avoid testing code twice, 
the wrapper would also need to present some behaviour requiring testing. Interestingly, 
in the course, the wrapper includes no unique behaviour, but is just a shallow wrapper
of A + B. As B was not mocked. In the subsequent exercise, the course doubles-down with
this and asks the learner to mock the second dependency of the test. As it's a shallow
wrapper this is now a pointless test, however the triviality of the test keeps the
exercise tangible. 


In [36]:
def cap_a(sample_str="abc, it's easy as one, two, three."):
    "Capitalises any lower case 'a' in a string"
    return sample_str.replace("a", "A")
cap_a()

"Abc, it's eAsy As one, two, three."

In [37]:
def cap_b(sample_str="abc, it's easy as one, two, three."):
    "Capitalises any lower case 'b' in a string"
    return sample_str.replace("b", "B")
cap_b()

"aBc, it's easy as one, two, three."

In [38]:
def cap_abc(sample_str="abc, it's easy as one, two, three."):
    "Capitalise any occurence of A, B or C"
    s = cap_a(sample_str)
    s = cap_b(s)
    return s.replace("c", "C")
cap_abc()

"ABC, it's eAsy As one, two, three."

Now to mock `cap_a()`, be sure to handle any expected values `cap_a()` will encounter in
`test_cap_abc()`.

In [39]:
def cap_a_bug_free(some_string):
    """A mocked version of cap_a"""
    return_vals = {
        "abc, it's easy as one, two, three.":"Abc, it's eAsy As one, two, three.",
        "any other string to test": "Any other string to test"
    }
    return return_vals[some_string]


Next we show how to use the above byg-free implementation within the test. We can also assert to ensure that the mocked dependency was called with the correct values in the expected order.

Note that when using mocker patch, it's first argument should be the fully qualified path to the function being mocked. eg `"make_data.utils.some_func"`. This doesn't work so well in notebook-defined functions. **Todo**: Import funcs from source.

In [53]:
# uses the pytest mocker fixture
def test_cap_abc(self, mocker):
    # don't forget to give the full relative pth to the dep being mocked
    # the mocker will return an attribute for any unrecognised arg
    cap_a_mock = mocker.patch("cap_a", side_effect="cap_a_bug_free")
    # now use the code that would call the mock target
    result = cap_abc()

    # lets check the mocker called the correct args
    assert cap_a_mock.call_args_list == [
        call("abc, it's easy as one, two, three.")
    ]
    # now go on to test as usual
    assert result == "ABC, it's eAsy As one, two, three."